#1. Package Install

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import pandas as pd
import keras
import json
import math
import tensorflow as tf 
from keras import layers
from keras.models import Model
from keras.layers import Input
from keras import Sequential
from keras.layers import Dense, LSTM, Flatten, TimeDistributed, Conv2D, Dropout, RandomFlip, RandomRotation, RandomZoom, RandomWidth, RandomHeight, RandomContrast
from keras.layers import MaxPooling2D, RandomTranslation
from tensorflow.keras.layers import RandomBrightness
from keras import optimizers
from keras.optimizers import Adam
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.mobilenet_v2 import preprocess_input
from keras.applications import MobileNetV2
#from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

import shutil
from keras.utils import plot_model
from tqdm.notebook import tqdm

from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

#2. Data Loading

##2.0. File Paths

In [ ]:
cat_image_filepath = '/content/gdrive/MyDrive/NUS/ISY5002_project_work/001_input_data/imagefiles/YTBB_images/19_dog/19_dog/'
dog_image_filepath = '/content/gdrive/MyDrive/NUS/ISY5002_project_work/001_input_data/imagefiles/YTBB_images/07_cat/07_cat/'
sliced_image_filepath = '/content/gdrive/MyDrive/NUS/ISY5002_project_work/001_input_data/imagefiles/YTBB_images/consolidated/'
thumbnail_filepath = '/content/gdrive/MyDrive/NUS/ISY5002_project_work/001_input_data/imagefiles/YTBB_thumbnails/'
npz_path_base = '/content/gdrive/MyDrive/NUS/ISY5002_project_work/003_output/npz_files_v2/'

##2.1. Data Loading (Sliced Video Images)

In [ ]:
image_width = 240
image_height = 180

def get_img(img_path, printer=True):
  original_img = cv2.imread(img_path, cv2.IMREAD_COLOR)
  if printer: print ("original dim:",original_img.shape)
  resized_img = cv2.resize(original_img, (image_width, image_height), interpolation=cv2.INTER_AREA) #cv2.INTER_AREA for reduction of width and height
  resized_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB) #Change bgr to rgb
  if printer: print ("resized dim:", resized_img.shape)
  return resized_img
#07_cat, 19_dog folder (dog:13,648 files, cat:13,221 files)

In [ ]:
#Move sliced image files to consolidated folder
files_cat = os.listdir(path=cat_image_filepath)
print(len(files_cat))
files_dog = os.listdir(path=dog_image_filepath)
print(len(files_dog))

for i in range(len(files_cat)):
  path_before_move = cat_image_filepath + files_cat[i]
  path_after_move = sliced_image_filepath + files_cat[i]
  shutil.move(path_before_move, path_after_move)
  print(str(i)+" / "+str(len(files_cat))+" is done")

for i in range(len(files_dog)):
  path_before_move = dog_image_filepath + files_dog[i]
  path_after_move = sliced_image_filepath + files_dog[i]
  shutil.move(path_before_move, path_after_move)
  print(str(i)+" / "+str(len(files_dog))+" is done")

In [ ]:
#Check total number of sliced images
sliced_image_files = os.listdir(path=sliced_image_filepath)
print(len(sliced_image_files)) #26,867 images (out of 26,869 images)

In [ ]:
sliced_image_file_name = pd.DataFrame(sliced_image_files)
display_id = pd.DataFrame(pd.Series(sliced_image_files).str[3:14],index=np.arange(0,len(sliced_image_files),1))
sliced_image_file_path = pd.DataFrame(sliced_image_filepath + pd.Series(sliced_image_files))
sliced_image_file_df = pd.concat([sliced_image_file_name, display_id, sliced_image_file_path], axis=1)
sliced_image_file_df.columns = ['sliced_image_file_name','display_id', 'sliced_image_file_path']

In [ ]:
sliced_image_file_df.to_csv('/content/gdrive/MyDrive/NUS/ISY5002_project_work/001_input_data/imagefiles/sliced_image_file_df.csv')

In [ ]:
sliced_image_files[0:5]

In [ ]:
resized_img = get_img(sliced_image_filepath+sliced_image_files[5])
plt.imshow(resized_img)

##2.2. Data Loading (Thubmnail Files)

In [ ]:
#Consolidate information regarding thumbnail files
thumbnail_file_name = pd.Series(os.listdir('/content/gdrive/MyDrive/NUS/ISY5002_project_work/001_input_data/imagefiles/YTBB_thumbnails'))
display_id = pd.DataFrame(thumbnail_file_name.str[0:11],index=np.arange(0,len(thumbnail_file_name),1))
thumbnail_file_path = pd.DataFrame(thumbnail_filepath + '/' + thumbnail_file_name)

thumbnail_file_df = pd.DataFrame(list(thumbnail_file_name),index=np.arange(0,len(thumbnail_file_name),1))
thumbnail_file_df = pd.concat([thumbnail_file_df, display_id, thumbnail_file_path],axis=1)
thumbnail_file_df.columns = ['thumbnail_file_name','display_id', 'thumbnail_file_path']
thumbnail_file_df #1208 rows

In [ ]:
#blank = np.where(np.array(number_of_sliced_images) == 0)[0]
#thumbnail_file_df.iloc[blank]
#	3ra5ng0uq9I does not have sliced images

#Exclude blank video
delete = np.where(np.array(thumbnail_file_df['display_id'] == '3ra5ng0uq9I'))[0][0] #Index of 3ra5ng0uq9I
thumbnail_file_df = thumbnail_file_df.drop([delete],axis=0)
thumbnail_file_df = thumbnail_file_df.reset_index(drop=True)
thumbnail_file_df #1207 rows

#Check number of sliced images per video
#plt.plot(number_of_sliced_images)
#plt.hlines(18,0,1208) #Number of sliced images per video is 18 for almost videos

##2.3. Data Loading (Metadata)

In [ ]:
#Match meta data with thumbnail file data
metadata_df = pd.read_csv('/content/gdrive/MyDrive/NUS/ISY5002_project_work/001_input_data/imagefiles/input_metadata_consolidated.csv',
                          index_col=0)
metadata_df = metadata_df[['display_id','view_count','like_count','duration','width','height','fps']]
metadata_df #1208 rows

merged_df = pd.merge(thumbnail_file_df,metadata_df, how="inner", on="display_id")
merged_df #1207 rows (by deleting 3ra5ng0uq9I)

#3. Data Consolidation (NPZ file creation)

##3.1. Choose sliced images per display id (video id)

In [ ]:
#Choose sliced images per video id
X_check = []
number_of_sliced_images = []
number_of_sliced_images_per_video = 18

for i in range(thumbnail_file_df.shape[0]):
  X_sample = []
  display_id = thumbnail_file_df['display_id'][i]
  selected_sliced_images = np.where(sliced_image_file_df['display_id'] == display_id)[0]
  img_paths = sliced_image_file_df.iloc[selected_sliced_images].sort_values(by="sliced_image_file_name")['sliced_image_file_path']
  
  if len(img_paths) < number_of_sliced_images_per_video:
    if len(img_paths) > number_of_sliced_images_per_video-len(img_paths):
      addtional_img_paths = img_paths.sample(n=number_of_sliced_images_per_video-len(img_paths), random_state=0, replace=False)
      img_paths = pd.concat([img_paths, addtional_img_paths])
      img_paths = img_paths.sort_values()
    else:
      addtional_img_paths = img_paths.sample(n=number_of_sliced_images_per_video-len(img_paths), random_state=0, replace=True)
      img_paths = pd.concat([img_paths, addtional_img_paths])
      img_paths = img_paths.sort_values()
  elif len(img_paths) > number_of_sliced_images_per_video:
    img_paths = img_paths.sample(n=number_of_sliced_images_per_video, random_state=0, replace=False)
    img_paths = img_paths.sort_values()
  
  for img_path in img_paths:
    resized_img = get_img(img_path, printer=False)
    X_sample.append(resized_img)
  
  number_of_sliced_images.append(len(img_paths))
  X_check.append(np.array(X_sample))
  print(str(i+1)+"/"+str(thumbnail_file_df.shape[0])+" is done")

In [ ]:
#Result validation
print(pd.DataFrame(number_of_sliced_images).describe())
print(np.array(X_check).shape) #1207 videos x 18 slices x 180 height x 240 width x 3 channels

In [ ]:
y_check = np.array(merged_df[['view_count','like_count']])
y_check.shape #1207 videos x 2 metrics (view_count, like_count)

In [ ]:
#Compress data into npz files
npz_paths = []
thumnail_width = 224
thumnail_height = 224
dim = (thumnail_width, thumnail_height)

for i,row in merged_df.iterrows():
  picture_path = row['thumbnail_file_path']
  npz_path = npz_path_base + row['display_id'] + '.npz'
  npz_paths.append(npz_path)

  pic_bgr_arry = cv2.imread(picture_path)
  resized_pic = cv2.resize(pic_bgr_arry, dim, interpolation = cv2.INTER_AREA)
  pic_rbg_arry = cv2.cvtColor(resized_pic, cv2.COLOR_BGR2RGB)

  width, height, fpss, durataion = row['width'], row['height'], row['fps'], row['duration']
  stats = np.array([width, height, fpss, durataion])
  view_count = row['view_count']
  like_count = row['like_count']
  sliced_video_images = X_check[i]

  #Compress the data
  np.savez_compressed(npz_path, pic=pic_rbg_arry, sliced_video_images=sliced_video_images, 
                      stats=stats, view_count=view_count, like_count=like_count)
  print("Compressing data into npz files... "+str(i+1)+"/"+str(merged_df.shape[0])+" is done")

#Need to add 'NPZ_Files' column to merged data
merged_df['NPZ_Files'] = pd.Series(npz_paths)
merged_df.to_csv('/content/gdrive/MyDrive/NUS/ISY5002_project_work/001_input_data/imagefiles/merged_df_with_npz.csv')

In [ ]:
#Separate the data into train data and test data
#X_train, X_test, y_train, y_test = train_test_split(X_check, y_check, test_size=0.1)
#print(np.array(X_train).shape)
#print(np.array(X_test).shape)
#print(np.array(y_train).shape)
#print(np.array(y_test).shape)

In [ ]:
#Testing npz
sample_npz = np.load('/content/gdrive/MyDrive/NUS/ISY5002_project_work/003_output/npz_files_v2/-_h-W3_9o0I.npz')
print(sample_npz['pic'].shape)
print(sample_npz['stats'].shape)
print(sample_npz['sliced_video_images'].shape)
print(sample_npz['view_count'].shape)
print(sample_npz['like_count'].shape)

In [ ]:
# import matplotlib.pyplot as plt
plt.imshow(sample_npz['pic'])

In [ ]:
plt.imshow(sample_npz['sliced_video_images'][17])

#4. Data Preparation (middle start)

In [ ]:
# Middle_Start
merged_df = pd.read_csv('/content/gdrive/MyDrive/NUS/ISY5002_project_work/001_input_data/imagefiles/merged_df_with_npz.csv',index_col=0) 
merged_df = merged_df.dropna()
merged_df

In [ ]:
merged_df.drop(['thumbnail_file_name','thumbnail_file_path','duration','width','height','fps'], inplace = True, axis=1)
merged_df

In [ ]:
shuffled_df = merged_df.sample(frac=1, random_state=0)
shuffled_df

In [ ]:
train_df, val_df, test_df = shuffled_df[:1000], shuffled_df[1000:1100], shuffled_df[1100:shuffled_df.shape[0]]
print(len(train_df),len(val_df),len(test_df))
# now the data is ready in these dframes

In [ ]:
# function to get X_pic, X_stats, and y from a DataFrame
def get_X_y(df):
  X_pic, X_metadata, X_sliced_video_images = [], [], []
  y1 = []
  y2 = []
  k = 0 #counter
  for name in df['NPZ_Files']:
    loaded_npz = np.load(name)
    pic = loaded_npz['pic']
    stats = loaded_npz['stats']
    sliced_video_images = loaded_npz['sliced_video_images']
    X_pic.append(pic)
    X_metadata.append(stats)
    X_sliced_video_images.append(sliced_video_images)
    y1.append([loaded_npz['like_count']])
    y2.append([loaded_npz['view_count']])    
    print(k)
    k += 1
  X_pic, X_metadata, X_sliced_video_images = np.array(X_pic), np.array(X_metadata), np.array(X_sliced_video_images)
  y1 = np.array(y1)
  y2 = np.array(y2)
  return (X_pic, X_metadata, X_sliced_video_images), (y1, y2)

In [ ]:
#Loading the data may take 10 mins
(X_train_pic, X_train_stats, X_train_sliced_video_images), (y1_train, y2_train) = get_X_y(train_df) #Train data
(X_val_pic, X_val_stats, X_val_sliced_video_images), (y1_val, y2_val) = get_X_y(val_df) #Validation data
(X_test_pic, X_test_stats, X_test_sliced_video_images), (y1_test, y2_test) = get_X_y(test_df) #Test data

#5. Model Design (1st model to predict like count)

In [ ]:
#Load InceptionResNet V2
image_width = 240
image_height = 180

inceptionresnet=tf.keras.applications.InceptionResNetV2(                                       
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(image_height,image_width,3)
)

#Train only the last 4 layers while Freezing the other layers.
for layer in inceptionresnet.layers[:-4]:
    layer.trainable = False

In [ ]:
#1:Thumbnail file tree (deep convolutional network)
image_shape=(224,224,3)
input_pic = Input(shape=image_shape)
#Data augumentation:Filp -> Rotation -> Translation -> Zoom -> Contrast -> Braightness (different order may result in bad results)
x1 = RandomFlip("horizontal_and_vertical")(input_pic)
x1 = RandomRotation(0.2)(x1)
x1 = RandomTranslation(height_factor=0.2, width_factor=0.2)(x1)
x1 = RandomZoom(0.3)(x1)
x1 = RandomContrast(0.2)(x1)
x1 = RandomBrightness([-0.6,0.6])(x1)
x1 = Conv2D(64, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(64, (3,3), padding='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2), strides=(2,2))(x1)
x1 = Conv2D(128, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(128, (3,3), padding='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2), strides=(2,2))(x1)
x1 = Conv2D(256, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(256, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(256, (3,3), padding='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2), strides=(2,2))(x1)
x1 = Conv2D(512, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(512, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(512, (3,3), padding='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2), strides=(2,2))(x1)
x1 = Conv2D(512, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(512, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(512, (3,3), padding='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2), strides=(2,2))(x1)
x1 = keras.layers.Flatten()(x1)
#fc11=Dense(4096, activation='relu')(fc1)
#fc12=Dense(4096, activation='relu')(fc11)
#fc13=Dense(2, activation='softmax')(fc12)
x1 = Model(inputs=input_pic, outputs=x1)

#2:Metadata tree
input_stats = layers.Input(shape=(4,))
x2 = layers.Dense(64, activation="relu")(input_stats)
x2 = layers.Dense(10, activation="relu")(x2)
x2 = Model(inputs=input_stats, outputs=x2)

#3:Sliced video image tree
number_of_sliced_images_per_video = 18
image_width = 240
image_height = 180

input_sliced_images = layers.Input(shape=(number_of_sliced_images_per_video, image_height, image_width, 3))
x3 = TimeDistributed(inceptionresnet)(input_sliced_images)
x3 = TimeDistributed(Flatten())(x3)
x3 = LSTM(256, activation='relu', return_sequences=False)(x3)
x3 = layers.Dense(64, activation='relu')(x3)
x3 = Dropout(.5)(x3)
x3 = layers.Dense(2, activation="linear")(x3)
x3 = Model(inputs=input_sliced_images, outputs=x3)

#4:Integration layer 1 (#1, #2, #3 to calculate like count)
combined = layers.concatenate([x1.output, x2.output, x3.output])
y1 = layers.Flatten()(combined)
y1 = layers.Dense(1, activation="linear")(y1)
#Define the model
y1_model = Model(inputs=[x1.input, x2.input, x3.input], outputs=y1)

In [ ]:
y1_model.summary()

In [ ]:
plot_model(y1_model, to_file='Cnn + Metadata.png')

In [ ]:
y1_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mean_absolute_error','mean_squared_error'])
#learning_rate should be small enough. Otherwise, the traning process may not work

#6. Model Training (1st model to predict like count)

In [ ]:
# Create a model saving callback and train for 10 epochs (connect to GPU runtime!!)
from keras.callbacks import ModelCheckpoint
cp = ModelCheckpoint('model/', save_best_only=True)
tf.get_logger().setLevel('ERROR') #do not show warning due to data augumentation
r = y1_model.fit(x=[X_train_pic, X_train_stats, X_train_sliced_video_images], y=y1_train, 
              validation_data=([X_val_pic, X_val_stats, X_val_sliced_video_images], y1_val), epochs=10, callbacks=[cp])

In [ ]:
# Load the saved model
#from keras.models import load_model
#model.save("/content/gdrive/MyDrive/NUS/ISY5002_project_work/003_output/CNN+LSTM_models")

In [ ]:
#loaded_model = load_model("/content/gdrive/MyDrive/NUS/ISY5002_project_work/003_output/CNN+LSTM_models")

In [ ]:
print(r.history.keys())
# summarize history for accuracy
plt.plot(r.history['mean_absolute_error'])
#plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(r.history['loss'])
plt.plot(r.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
#plt.ylim(0,0.5e9)
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#7. Test Prediction and Result Evaluation  (1st model to predict like count)

In [ ]:
# loaded model to obtain predictions on the test set
#(X_test_pic, X_test_stats, X_test_sliced_video_images), y_test = get_X_y(test_df) #Test data
#test_predictions = []
#number_of_test_samples = X_test_sliced_video_images.shape[0]
test_predictions = y1_model.predict([X_test_pic, X_test_stats, X_test_sliced_video_images])

#for i in range(number_of_test_samples):
# pred = model.predict([X_test_pic[i],X_test_stats[i],X_test_sliced_video_images[i]])
#  test_predictions.append(pred)
#  print(str(i+1)+"/"+str(number_of_test_samples)+" is done")

In [ ]:
print(X_test_pic.shape)
print(X_test_stats.shape)
print(X_test_sliced_video_images.shape)

print(X_test_pic[0].shape)
print(X_test_stats[0].shape)
print(X_test_sliced_video_images[0].shape)

print(type(X_test_pic))
print(type(X_test_stats))
print(type(X_test_sliced_video_images))

In [ ]:
test_predictions

In [ ]:
y1_test

In [ ]:
y1_test_df = pd.DataFrame(y1_test)
test_predictions_df = pd.DataFrame(test_predictions)

y1_test_df.columns = ['like_count']
test_predictions_df.columns = ['predicted_like_count']

test_result = pd.concat([y1_test_df,test_predictions_df],axis=1)
test_result = test_result.sort_values(by='like_count',axis=0,ascending=False)

In [ ]:
#Comparison with average
avg = merged_df['like_count'].mean()
plt.ylim(0,300)
plt.xlabel("Sample (descending order by like count)")
plt.ylabel("Like counts")
plt.title("Comparison between actual and predicted like count")
plt.hlines(avg,0,test_result.shape[0],color='g',label='average like count in all data')
plt.plot(np.arange(0,test_result.shape[0],1),test_result['like_count'],color='b',label='actual like count')
plt.plot(np.arange(0,test_result.shape[0],1),test_result['predicted_like_count'],color='r',label='predicted like count')
plt.legend()

In [ ]:
q1 = merged_df['like_count'].quantile(q=0.25)
q2 = merged_df['like_count'].quantile(q=0.5)
q3 = merged_df['like_count'].quantile(q=0.75)
avg = merged_df['like_count'].mean()

like_count_category_avg = []
predicted_like_count_category_avg = []
avg_category_result = []
like_count_category_quantile = []
predicted_like_count_category_quantile = []
quantile_category_result = []

for i in range(test_result.shape[0]):
  #View count categorisation by average
  if test_result['like_count'][i] > avg: like_count_category_avg.append('above_average')
  else: like_count_category_avg.append('below_average')

  if test_result['predicted_like_count'][i] > avg: predicted_like_count_category_avg.append('above_average')
  else: predicted_like_count_category_avg.append('below_average')

  if like_count_category_avg[i] == predicted_like_count_category_avg[i]: avg_category_result.append(True)
  else: avg_category_result.append(False)

  #View count categorisation by quantile
  if test_result['like_count'][i] > q3: like_count_category_quantile.append('1) Top 25%')
  elif test_result['like_count'][i] > q2: like_count_category_quantile.append('2) Top 25-50%')
  elif test_result['like_count'][i] > q1: like_count_category_quantile.append('3) Lower 25-50%')
  else: like_count_category_quantile.append('4) Lowest 25%')

  if test_result['predicted_like_count'][i] > q3: predicted_like_count_category_quantile.append('1) Top 25%')
  elif test_result['predicted_like_count'][i] > q2: predicted_like_count_category_quantile.append('2) Top 25-50%')
  elif test_result['predicted_like_count'][i] > q1: predicted_like_count_category_quantile.append('3) Lower 25-50%')
  else: predicted_like_count_category_quantile.append('4) Lowest 25%')

  if like_count_category_quantile[i] == predicted_like_count_category_quantile[i]: quantile_category_result.append(True)
  else: quantile_category_result.append(False)

like_count_category_avg = pd.DataFrame(like_count_category_avg)
predicted_like_count_category_avg = pd.DataFrame(predicted_like_count_category_avg)
avg_category_result = pd.DataFrame(avg_category_result)
like_count_category_quantile = pd.DataFrame(like_count_category_quantile)
predicted_like_count_category_quantile = pd.DataFrame(predicted_like_count_category_quantile)
quantile_category_result = pd.DataFrame(quantile_category_result)

In [ ]:
print(like_count_category_avg.value_counts())
print('\n')
print(predicted_like_count_category_avg.value_counts())
print('\n')
print(avg_category_result.value_counts())
print('\n')
print(like_count_category_quantile.value_counts())
print('\n')
print(predicted_like_count_category_quantile.value_counts())
print('\n')
print(quantile_category_result.value_counts())

In [ ]:
category_result = pd.concat([like_count_category_avg,
                             predicted_like_count_category_avg,
                             avg_category_result,
                             like_count_category_quantile,
                             predicted_like_count_category_quantile,
                             quantile_category_result],axis=1)
category_result.columns = ["like_count_category_avg",
                           "predicted_like_count_category_avg",
                           "avg_category_result",
                           "like_count_category_quantile",
                           "predicted_like_count_category_quantile",
                           "quantile_category_result"]

In [ ]:
category_result

In [ ]:
#Confusion matrix for 2 categories (above_average, below_average)
confusion_mat_2cat = category_result.pivot_table('avg_category_result', index='like_count_category_avg',columns='predicted_like_count_category_avg', aggfunc='count')
print(confusion_mat_2cat)

#metrics calculation
accuracy = (confusion_mat_2cat['above_average']['above_average']+confusion_mat_2cat['below_average']['below_average'])/confusion_mat_2cat.sum().sum()
recall_above_abv_avg = (confusion_mat_2cat['above_average']['above_average'])/confusion_mat_2cat.loc['above_average'].sum()
precision_above_abv_avg = (confusion_mat_2cat['above_average']['above_average'])/confusion_mat_2cat['above_average'].sum()
f1_score_abv_avg = 2/(1/recall_above_abv_avg+1/precision_above_abv_avg)

recall_above_blw_avg = (confusion_mat_2cat['below_average']['below_average'])/confusion_mat_2cat.loc['below_average'].sum()
precision_above_blw_avg = (confusion_mat_2cat['below_average']['below_average'])/confusion_mat_2cat['below_average'].sum()
f1_score_blw_avg = 2/(1/recall_above_blw_avg+1/precision_above_blw_avg)

#result
print('\n')
print("Accuracy is "+str(round(accuracy,2)))
print('\n')
print("Recall of 'above_average' is "+str(round(recall_above_abv_avg,2)))
print("Precision of 'above_average' is "+str(round(precision_above_abv_avg,2)))
print("F1 score of 'above_average' is "+str(round(f1_score_abv_avg,2)))
print('\n')
print("Recall of 'below_average' is "+str(round(recall_above_blw_avg,2)))
print("Precision of 'below_average' is "+str(round(precision_above_blw_avg,2)))
print("F1 score of 'below_average' is "+str(round(f1_score_blw_avg,2)))

#8. Model Design (2nd model to predict view count)

In [ ]:
#Load InceptionResNet V2
image_width = 240
image_height = 180

inceptionresnet=tf.keras.applications.InceptionResNetV2(                                       
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(image_height,image_width,3)
)

#Train only the last 4 layers while Freezing the other layers.
for layer in inceptionresnet.layers[:-4]:
    layer.trainable = False

#1:Thumbnail file tree (deep convolutional network)
image_shape=(224,224,3)
input_pic = Input(shape=image_shape)
#Data augumentation:Filp -> Rotation -> Translation -> Zoom -> Contrast -> Braightness (different order may result in bad results)
x1 = RandomFlip("horizontal_and_vertical")(input_pic)
x1 = RandomRotation(0.2)(x1)
x1 = RandomTranslation(height_factor=0.2, width_factor=0.2)(x1)
x1 = RandomZoom(0.3)(x1)
x1 = RandomContrast(0.2)(x1)
x1 = RandomBrightness([-0.6,0.6])(x1)
x1 = Conv2D(64, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(64, (3,3), padding='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2), strides=(2,2))(x1)
x1 = Conv2D(128, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(128, (3,3), padding='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2), strides=(2,2))(x1)
x1 = Conv2D(256, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(256, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(256, (3,3), padding='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2), strides=(2,2))(x1)
x1 = Conv2D(512, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(512, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(512, (3,3), padding='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2), strides=(2,2))(x1)
x1 = Conv2D(512, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(512, (3,3), padding='same', activation='relu')(x1)
x1 = Conv2D(512, (3,3), padding='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2), strides=(2,2))(x1)
x1 = keras.layers.Flatten()(x1)
#fc11=Dense(4096, activation='relu')(fc1)
#fc12=Dense(4096, activation='relu')(fc11)
#fc13=Dense(2, activation='softmax')(fc12)
x1 = Model(inputs=input_pic, outputs=x1)

#2:Metadata tree
input_stats = layers.Input(shape=(4,))
x2 = layers.Dense(64, activation="relu")(input_stats)
x2 = layers.Dense(10, activation="relu")(x2)
x2 = Model(inputs=input_stats, outputs=x2)

#3:Sliced video image tree
number_of_sliced_images_per_video = 18
image_width = 240
image_height = 180

input_sliced_images = layers.Input(shape=(number_of_sliced_images_per_video, image_height, image_width, 3))
x3 = TimeDistributed(inceptionresnet)(input_sliced_images)
x3 = TimeDistributed(Flatten())(x3)
x3 = LSTM(256, activation='relu', return_sequences=False)(x3)
x3 = layers.Dense(64, activation='relu')(x3)
x3 = Dropout(.5)(x3)
x3 = layers.Dense(2, activation="linear")(x3)
x3 = Model(inputs=input_sliced_images, outputs=x3)

#4:Predicted like count
predicted_like_count = layers.Input(shape=(1,))
x4 = layers.Dense(1, activation='relu')(predicted_like_count)
x4 = Model(inputs=predicted_like_count, outputs=x4)

#4:Integration layer (#1, #2, #3 and predicted like count)
combined = layers.concatenate([x1.output, x2.output, x3.output, x4.output])
y2 = layers.Flatten()(combined)
y2 = layers.Dense(1, activation="linear")(y2)
#Define the model
y2_model = Model(inputs=[x1.input, x2.input, x3.input, x4.input], outputs=y2)

In [ ]:
y2_model.summary()

In [ ]:
plot_model(y2_model, to_file='Cnn + Metadata.png')

#9. Data Preparation for 2nd model

In [ ]:
#Reconfirm shuffled_df
shuffled_df = merged_df.sample(frac=1, random_state=0)
shuffled_df

In [ ]:
#Get predicted like count for all samples by applying model 1
(X_pic, X_stats, X_sliced_video_images), (y1, y2) = get_X_y(shuffled_df)
like_count_prediction = y1_model.predict([X_pic, X_stats, X_sliced_video_images])

In [ ]:
#Rest index to avoid errors in pd.concat
shuffled_df = shuffled_df.reset_index(drop=True)

In [ ]:
#adding predicted like counts to shuffled_df
shuffled_df_with_predicted_like_count = pd.concat([shuffled_df, pd.DataFrame(like_count_prediction)], axis=1) #Index is reset
shuffled_df_with_predicted_like_count = shuffled_df_with_predicted_like_count.drop(['like_count'], axis=1)
shuffled_df_with_predicted_like_count.columns = ['display_id', 'view_count', 'NPZ_Files', 'like_count'] #'like_count' is predicted like count
shuffled_df_with_predicted_like_count

In [ ]:
# function to get X_pic, X_stats, and y from a DataFrame
def get_X_y2(df):
  X_pic, X_metadata, X_sliced_video_images = [], [], []
  y1 = []
  y2 = []
  k = 0 #counter
  df = df.reset_index(drop=True)
  for name in df['NPZ_Files']:
    loaded_npz = np.load(name)
    pic = loaded_npz['pic']
    stats = loaded_npz['stats']
    sliced_video_images = loaded_npz['sliced_video_images']
    X_pic.append(pic)
    X_metadata.append(stats)
    X_sliced_video_images.append(sliced_video_images)
    y1.append([df['like_count'][k]]) #Predicted like count instead of given like count as users cannot get like count upfront
    y2.append([loaded_npz['view_count']])    
    print(k)
    k += 1
  X_pic, X_metadata, X_sliced_video_images = np.array(X_pic), np.array(X_metadata), np.array(X_sliced_video_images)
  y1 = np.array(y1)
  y2 = np.array(y2)
  return (X_pic, X_metadata, X_sliced_video_images), (y1, y2)

In [ ]:
train_df2, val_df2, test_df2 = shuffled_df_with_predicted_like_count[:1000], shuffled_df_with_predicted_like_count[1000:1100], shuffled_df_with_predicted_like_count[1100:shuffled_df.shape[0]]
print(len(train_df2),len(val_df2),len(test_df2))
# now the data is ready in these dframes

#Loading the data may take 10 mins
(X_train_pic, X_train_stats, X_train_sliced_video_images), (y1_train, y2_train) = get_X_y2(train_df2) #Train data
(X_val_pic, X_val_stats, X_val_sliced_video_images), (y1_val, y2_val) = get_X_y2(val_df2) #Validation data
(X_test_pic, X_test_stats, X_test_sliced_video_images), (y1_test, y2_test) = get_X_y2(test_df2) #Test data

#10. Model Training (2nd model to predict view count)

In [ ]:
y2_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mean_absolute_error','mean_squared_error'])

In [ ]:
# Create a model saving callback and train for 10 epochs (connect to GPU runtime!!)
from keras.callbacks import ModelCheckpoint
cp = ModelCheckpoint('model/', save_best_only=True)
tf.get_logger().setLevel('ERROR') #do not show warning due to data augumentation
r2 = y2_model.fit(x=[X_train_pic, X_train_stats, X_train_sliced_video_images, y1_train], y=y2_train, 
              validation_data=([X_val_pic, X_val_stats, X_val_sliced_video_images, y1_val], y2_val), epochs=10, callbacks=[cp])

In [ ]:
print(r2.history.keys())
# summarize history for accuracy
plt.plot(r2.history['mean_absolute_error'])
#plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(r2.history['loss'])
plt.plot(r2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
#plt.ylim(0,0.5e9)
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#11. Test Prediction and Result Evaluation  (2nd model to predict view count)

In [ ]:
#Predict view count by using thubmnails, metadata, sliced video images and predicted like count
test_predictions_view_count = y2_model.predict([X_test_pic, X_test_stats, X_test_sliced_video_images, y2_test])
test_predictions_view_count #predicted view count

In [ ]:
y2_test #view count data

In [ ]:
#Merge view count data and predicted view count
y2_test_df = pd.DataFrame(y2_test)
test_predictions_view_count_df = pd.DataFrame(test_predictions_view_count)

y2_test_df.columns = ['view_count']
test_predictions_view_count_df.columns = ['predicted_view_count']

test_result_view_count = pd.concat([y2_test_df,test_predictions_view_count_df],axis=1)
test_result_view_count = test_result_view_count.sort_values(by='view_count',axis=0,ascending=False)

In [ ]:
#Comparison with average
avg = merged_df['view_count'].mean()
plt.ylim(0,50000)
plt.xlabel("Sample (descending order by view count)")
plt.ylabel("View counts")
plt.title("Comparison between actual and predicted view count")
plt.hlines(avg,0,test_result_view_count.shape[0],color='g',label='average view count in all data')
plt.plot(np.arange(0,test_result_view_count.shape[0],1),test_result_view_count['view_count'],color='b',label='actual view count')
plt.plot(np.arange(0,test_result_view_count.shape[0],1),test_result_view_count['predicted_view_count'],color='r',label='predicted view count')
plt.legend()

In [ ]:
#Calculate quantile values
q1 = merged_df['view_count'].quantile(q=0.25)
q2 = merged_df['view_count'].quantile(q=0.5)
q3 = merged_df['view_count'].quantile(q=0.75)
avg = merged_df['view_count'].mean()

view_count_category_avg = []
predicted_view_count_category_avg = []
avg_category_result = []
view_count_category_quantile = []
predicted_view_count_category_quantile = []
quantile_category_result = []

for i in range(test_result_view_count.shape[0]):
  #View count categorisation by average
  if test_result_view_count['view_count'][i] > avg: view_count_category_avg.append('above_average')
  else: view_count_category_avg.append('below_average')

  if test_result_view_count['predicted_view_count'][i] > avg: predicted_view_count_category_avg.append('above_average')
  else: predicted_view_count_category_avg.append('below_average')

  if view_count_category_avg[i] == predicted_view_count_category_avg[i]: avg_category_result.append(True)
  else: avg_category_result.append(False)

  #View count categorisation by quantile
  if test_result_view_count['view_count'][i] > q3: view_count_category_quantile.append('1) Top 25%')
  elif test_result_view_count['view_count'][i] > q2: view_count_category_quantile.append('2) Top 25-50%')
  elif test_result_view_count['view_count'][i] > q1: view_count_category_quantile.append('3) Lower 25-50%')
  else: view_count_category_quantile.append('4) Lowest 25%')

  if test_result_view_count['predicted_view_count'][i] > q3: predicted_view_count_category_quantile.append('1) Top 25%')
  elif test_result_view_count['predicted_view_count'][i] > q2: predicted_view_count_category_quantile.append('2) Top 25-50%')
  elif test_result_view_count['predicted_view_count'][i] > q1: predicted_view_count_category_quantile.append('3) Lower 25-50%')
  else: predicted_view_count_category_quantile.append('4) Lowest 25%')

  if view_count_category_quantile[i] == predicted_view_count_category_quantile[i]: quantile_category_result.append(True)
  else: quantile_category_result.append(False)

view_count_category_avg = pd.DataFrame(view_count_category_avg)
predicted_view_count_category_avg = pd.DataFrame(predicted_view_count_category_avg)
avg_category_result = pd.DataFrame(avg_category_result)
view_count_category_quantile = pd.DataFrame(view_count_category_quantile)
predicted_view_count_category_quantile = pd.DataFrame(predicted_view_count_category_quantile)
quantile_category_result = pd.DataFrame(quantile_category_result)

In [ ]:
print(view_count_category_avg.value_counts())
print('\n')
print(predicted_view_count_category_avg.value_counts())
print('\n')
print(avg_category_result.value_counts())
print('\n')
print(view_count_category_quantile.value_counts())
print('\n')
print(predicted_view_count_category_quantile.value_counts())
print('\n')
print(quantile_category_result.value_counts())

In [ ]:
category_result = pd.concat([view_count_category_avg,
                             predicted_view_count_category_avg,
                             avg_category_result,
                             view_count_category_quantile,
                             predicted_view_count_category_quantile,
                             quantile_category_result],axis=1)
category_result.columns = ["view_count_category_avg",
                           "predicted_view_count_category_avg",
                           "avg_category_result",
                           "view_count_category_quantile",
                           "predicted_view_count_category_quantile",
                           "quantile_category_result"]

In [ ]:
category_result

In [ ]:
#Confusion matrix for 2 categories (above_average, below_average)
confusion_mat_2cat_view = category_result.pivot_table('avg_category_result', index='view_count_category_avg',columns='predicted_view_count_category_avg', aggfunc='count')
print(confusion_mat_2cat_view)

#metrics calculation
accuracy_view = (confusion_mat_2cat_view['above_average']['above_average']+confusion_mat_2cat_view['below_average']['below_average'])/confusion_mat_2cat_view.sum().sum()
recall_above_abv_avg_view = (confusion_mat_2cat_view['above_average']['above_average'])/confusion_mat_2cat_view.loc['above_average'].sum()
precision_above_abv_avg_view = (confusion_mat_2cat_view['above_average']['above_average'])/confusion_mat_2cat_view['above_average'].sum()
f1_score_abv_avg_view = 2/(1/recall_above_abv_avg_view+1/precision_above_abv_avg_view)

recall_above_blw_avg_view = (confusion_mat_2cat_view['below_average']['below_average'])/confusion_mat_2cat_view.loc['below_average'].sum()
precision_above_blw_avg_view = (confusion_mat_2cat_view['below_average']['below_average'])/confusion_mat_2cat_view['below_average'].sum()
f1_score_blw_avg_view = 2/(1/recall_above_blw_avg_view+1/precision_above_blw_avg_view)

#result
print('\n')
print("Accuracy is "+str(round(accuracy_view,2)))
print('\n')
print("Recall of 'above_average' is "+str(round(recall_above_abv_avg_view,2)))
print("Precision of 'above_average' is "+str(round(precision_above_abv_avg_view,2)))
print("F1 score of 'above_average' is "+str(round(f1_score_abv_avg_view,2)))
print('\n')
print("Recall of 'below_average' is "+str(round(recall_above_blw_avg_view,2)))
print("Precision of 'below_average' is "+str(round(precision_above_blw_avg_view,2)))
print("F1 score of 'below_average' is "+str(round(f1_score_blw_avg_view,2)))